### 재무정보(전체 재무제표) - 단일 회사(상장기업) - 분기 및 반기 보고서

In [40]:
fiscal_year = '2020'
listed_corp_codes = \
['01208885',
 '00787376',
 '00128555',
 '00130684']
report_codes = \
['11013',   # 1분기보고서
 '11012',   # 반기보고서
 '11014']   # 3분기보고서

In [41]:
# 재무제표 가져오기
import requests
import pandas as pd

url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
df_fi_all = pd.DataFrame()

for corp_code in listed_corp_codes:
    for rpt_code in report_codes:
        params = {
            'corp_code': corp_code,
            'bsns_year': fiscal_year,
            'reprt_code': rpt_code,
            'fs_div': 'OFS',            # OFS:재무제표
            'crtfc_key': api_key,
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:             # URL GET 200 : 정상
            print("URL GET Error")
        if response.json()['status'] == '000':      # 데이터 유무 '000' : 정상
            # 데이터프레임 형태로 변환
            df_fi = pd.json_normalize(response.json()['list'])
            if not df_fi.empty:
                df_fi_all = pd.concat([df_fi_all, df_fi], ignore_index=True)

In [42]:
df_fi_all.groupby(['corp_code', 'reprt_code']).agg(n = ('account_nm', 'count'))

n
corp_code reprt_code     
00128555  11012       136
          11013       135
          11014       139
00130684  11012       167
          11013       162
          11014       168
00787376  11012       160
          11013       158
          11014       166
01208885  11012       128
          11013       118
          11014       130

In [43]:
df_fi_all.dtypes

rcept_no             object
reprt_code           object
bsns_year            object
corp_code            object
sj_div               object
sj_nm                object
account_id           object
account_nm           object
account_detail       object
thstrm_nm            object
thstrm_amount        object
frmtrm_nm            object
frmtrm_amount        object
ord                  object
currency             object
thstrm_add_amount    object
frmtrm_q_nm          object
frmtrm_q_amount      object
frmtrm_add_amount    object
dtype: object

In [44]:
# df_fi_all.to_excel('D:\PythonProject\data-gatherer\dart\co_fi_all_multiple.xlsx')

In [45]:
# df_fi_all['account_detail'].str.len().max()
# df_fi_all['thstrm_nm'].str.len().max()
# df_fi_all['frmtrm_nm'].str.len().max()
# df_fi_all['bfefrmtrm_nm'].str.len().max()


In [46]:
# type = '자본변동표'
# df_fi_all.query('sj_nm == @type')

In [47]:
# 불필요한 컬럼 삭제
df_fi_all.drop(labels=['account_id', 'thstrm_add_amount', \
                       'frmtrm_q_nm', 'frmtrm_q_amount', 'frmtrm_add_amount'], axis=1, inplace=True)

In [48]:
# df_fi_all.dtypes

In [49]:
df_fi_all.shape

(1767, 14)

In [50]:
# df_fi_all.isna().value_counts()

In [51]:
# 문자열 컬럼에 있는 null값을 ''로 변환
df_fi_all['thstrm_nm'].fillna('', inplace=True)
df_fi_all['frmtrm_nm'].fillna('', inplace=True)

# 금액 컬럼에 있는 null값을 ''로 변환
df_fi_all['thstrm_amount'].fillna('', inplace=True)
df_fi_all['frmtrm_amount'].fillna('', inplace=True)
# 금액 컬럼에 있는 ''을 '0'로 변환(저장 시에 숫자 형식으로 변환을 위해서임)
df_fi_all = df_fi_all.replace({'thstrm_amount' : '',
                               'frmtrm_amount' : ''}, '0')
# 타입 변환
df_fi_all = df_fi_all.astype({'ord' : 'int'})
df_fi_all = df_fi_all.astype({'thstrm_amount' : 'float',
                              'frmtrm_amount' : 'float'})

### 테이블에 저장

In [52]:
# 테이블 컬럼 순서대로 항목 재정렬
df_fi_save = df_fi_all[['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm', 'account_nm', 'account_detail', \
                       'thstrm_nm', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'ord', 'currency']]

In [53]:
df_fi_save.shape

(1767, 14)

In [54]:
# df_fi_save.to_excel('D:\PythonProject\data-gatherer\dart\df_fi_save.xlsx')

In [55]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fi_save.itertuples():
    sql = "insert into dart_report_detail(rcept_no, reprt_code, bsns_year, corp_code, sj_div, sj_nm, account_nm, account_detail, \
                                        thstrm_nm, thstrm_amount, frmtrm_nm, frmtrm_amount, \
                                        ord, currency) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], \
                    row[9], row[10], row[11], row[12], row[13], row[14]))

conn.commit()
conn.close()